# Homework 2 - Recurrent Neural Networks


In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.


## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.


In [215]:
import random
import string

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
    """ Converts the generated integers to one-hot vectors """
    ones = torch.sparse.torch.eye(k)
    shape = x.shape
    res = ones.index_select(0, x.view(-1).type(torch.int64))
    return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, delay=4, seq_length=15, size=1000):
        self.delay = delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        """ Iterable dataset doesn't have to implement __getitem__.
            Instead, we only need to implement __iter__ to return
            an iterator (or generator).
        """
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1))
                               for i in range(self.seq_length)], dtype=torch.int64)
            result = torch.cat(
                (torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
            yield seq, result


DELAY = 4
DATASET_SIZE = 200000
MAX_EPOCHS = 500
BATCH_SIZE = 1000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)
train_ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)
val_ds = EchoDataset(delay=DELAY, size=5000)


## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.


In [216]:
in_dim = N+1
out_dim = N+1
num_layers = 2  # stack?


class GRUMemory(torch.nn.Module):

    def __init__(self, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, 12)
        self.fc2 = nn.Linear(hidden_size, out_dim)
        self.relu = nn.ReLU()
        self.gru = nn.GRU(12, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
        x = self.fc1(x)
        x, _ = self.gru(x)  # no hidden; h0 assumed 0s
        x = self.relu(x)
        x = self.fc2(x)
        return x

    @torch.no_grad()
    def test_run(self, s):
        # This function accepts one string s containing lowercase characters a-z.
        # You need to map those characters to one-hot encodings,
        # then get the result from your network, and then convert the output
        # back to a string of the same length, with 0 mapped to ' ',
        # and 1-26 mapped to a-z.

        # one-hot
        alpha_num = torch.from_numpy(
            np.array([ord(i) - 96 if ord(i) >= 97 else ord(i) - 32 for i in list(s)]))
        encoding = idx_to_onehot(alpha_num, k=N + 1).unsqueeze(0).to(device)

        logits = self.forward(encoding)
        predict = torch.argmax(logits, dim=2)

        return [chr(x + 96) if x >= 1 else chr(x + 32) for x in predict[0, :]]


## Training

Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.


In [217]:
def test_model(model, sequence_length=15):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase)
                    for i in range(random.randint(15, 25))])
        result = model.test_run(s)
        for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
            correct += int(c1 == c2)
        total += len(s) - DELAY

    return correct / total


In [218]:
# implement usual training loop
def train(batch, optimizer, criterion, device):
    model.train()
    num_correct = 0

    data = idx_to_onehot(batch[0], k = N+1).to(device)
    target = batch[1].to(device)

    output = model(data)
    
    prediction = torch.argmax(output, dim = 2)
    output = output.transpose(2,1)
    loss = criterion(output, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    matches = (prediction == target)
    num_total = matches.size(0) * matches.size(1)
    num_correct = matches.sum()
    
    return num_correct, num_total
    

In [219]:
# implement usual evaluation loop
def eval(batch, optimizer, criterion, device):
    model.eval()
    num_correct = 0

    data = idx_to_onehot(batch[0], k = N+1).to(device)
    target = batch[1].to(device)

    output = model(data)
    predictions = torch.argmax(output, dim = 2)
    
    matches = (predictions == target)
    num_total = matches.size(0) * matches.size(1)
    num_correct = matches.sum()
    
    return num_correct, num_total
                          

In [220]:
import time
MAX_EPOCHS = 30
BATCH_SIZE = 1000
train_dataloader = torch.utils.data.DataLoader(
    train_ds, batch_size=BATCH_SIZE, num_workers=0)  # windows no like num workers
val_dataloader = torch.utils.data.DataLoader(
    val_ds, batch_size=BATCH_SIZE, num_workers=0)

criterion = nn.CrossEntropyLoss()
model = GRUMemory(246)
optimizer = torch.optim.Adam(model.parameters(), 0.005)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    device = torch.device("cuda:0")
    print("device: {}".format(device))
else:
    device = torch.device("cpu")

model.to(device)
criterion = criterion.to(device)

total_val_losses = []
total_val_accuracies = []
total_train_losses = []
total_train_accuracies = []

start_time = time.time()

progress = tqdm(range(MAX_EPOCHS))
for i in progress:
    total_train_correct = 0
    total_train = 0
    total_val_correct = 0.0
    total_val = 0

    for i, batch in enumerate(train_dataloader):
        batch_correct, batch_total = train(batch, optimizer, criterion, device)
        total_train_correct += batch_correct
        total_train += batch_total

    with torch.no_grad():
        for i, batch in enumerate(val_dataloader):
            batch_correct, batch_total = eval(
                batch, optimizer, criterion, device)
            total_val_correct += batch_correct
            total_val += batch_total

    train_accuracy = total_train_correct/total_train
    val_accuracy = total_val_correct/total_val
    progress.set_postfix({'train_accuracy': train_accuracy,
                         'validation_accuracy': val_accuracy})

    # EARLY STOP
    if val_accuracy > 0.99:
        test_accuracy = test_model(model)
        if test_accuracy >= 0.99:
            break

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')


device: cuda:0


  0%|          | 0/30 [00:00<?, ?it/s]

tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.


### Dataset

As before, we first implement the dataset:


In [226]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

train_delayed = VariableDelayEchoDataset(size = DATASET_SIZE)
val_delayed = VariableDelayEchoDataset(size = 5000)

### Model

And the model.


In [222]:
in_dim = N + 2
out_dim = N + 2
num_layers = 2

class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    self.max_delay = max_delay
    self.fc1 = nn.Linear(in_dim, 12)
    self.gru = nn.GRU(12, hidden_size, num_layers, batch_first=True)
    self.fc2 = nn.Linear(hidden_size, 60)
    self.fc3 = nn.Linear(60, out_dim)
    self.relu = nn.ReLU()

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    delays = delays[:, None, None].expand(-1,20,-1)

    x = idx_to_onehot(x, k = N+1)
    if delays.size(0) <= 1:
      x = x.unsqueeze(0)

    x = torch.cat((x,delays), dim = 2).to(device)
    x, _ = self.gru(self.fc1(x))
    x = self.relu(self.fc2(x))
    return self.fc3(x)
    

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    alpha_num = torch.from_numpy(np.array([ord(i) - 96 if ord(i) >= 97 else ord(i) - 32 for i in list(s)]))

    logits = self.forward(alpha_num, torch.tensor(delay).reshape(1))
    predict = torch.argmax(logits, dim = 2)
    return [chr(x + 96) if x>=1 else chr(x+ 32) for x in predict[0, :]]
    


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.


In [223]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [224]:
# implement usual training loop
def train_delay(batch, optimizer, criterion):
    model.train()
    n_correct = 0
    n_total = 0

    data, delay, target = batch[0], batch[1], batch[2]
    target = target.to(device)

    output= model(data, delay)
    
    predictions = torch.argmax(output, dim = 2)
    output = output.transpose(2,1)
    loss = criterion(output, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    matches = (predictions == target)
    n_correct = matches.sum()
    n_total = matches.size(0)*matches.size(1)
    
    return n_correct, n_total, predictions

	# implement usual evaluation loop
def eval_delay(batch, optimizer, criterion):
    model.eval()
    n_correct = 0
    n_total = 0

    data, delay, target = batch[0], batch[1], batch[2]
    target = target.to(device)

    output = model(data, delay)
    predictions = torch.argmax(output, dim = 2) 
    output = output.transpose(2,1)
    
    matches = (predictions == target)
    n_correct = matches.sum()
    n_total = matches.size(0)*matches.size(1)
    
    return n_correct, n_total, predictions

In [229]:
import time
start_time = time.time()

MAX_DELAY = 8
BATCH_SIZE = 500

train_delay_dl = torch.utils.data.DataLoader(train_delayed, batch_size = BATCH_SIZE, num_workers = 0)
val_delay_dl = torch.utils.data.DataLoader(val_delayed, batch_size = BATCH_SIZE, num_workers = 0)

model = VariableDelayGRUMemory(200, MAX_DELAY)
optimizer = torch.optim.Adam(model.parameters(), lr=.0008)
criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
  model = model.cuda()
  criterion = criterion.cuda()
  device = torch.device("cuda:0") 
  print("device: {}".format(device))
else:
  device = torch.device("cpu")

model.to(device)
criterion = criterion.to(device)

progress = tqdm(range(1000))

for i in progress:
  total_train_correct = 0
  total_train = 0
  total_val_correct = 0.0
  total_val = 0

  for i, batch in enumerate(train_delay_dl):
      batch_correct, batch_total, preds = train_delay(batch, optimizer, criterion)
      total_train_correct += batch_correct
      total_train += batch_total

  with torch.no_grad():
      for i, batch in enumerate(val_delay_dl):
          batch_correct, batch_total = eval_delay(batch, optimizer, criterion)
          total_val_correct += batch_correct
          total_val += batch_total
  
  train_accuracy = total_train_correct/total_train
  val_accuracy = total_val_correct/total_val
  
  if val_accuracy > 0.99:
    test_accuracy = test_variable_delay_model(model)
    if test_accuracy >= 0.99:
        break

  progress.set_postfix({'train_accuracy': train_accuracy, 'validation_accuracy': val_accuracy})

end_tim = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

device: cuda:0


  0%|          | 0/1000 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)